In [16]:
import torch

In [17]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import random_split

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)

train_dataset, val_dataset = random_split(dataset = dataset, lengths = [int(0.8 * len(dataset)), int(0.2 * len(dataset))])
test_dataset = datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)

In [18]:
from torch.utils.data import DataLoader

batch_size = 64

train_dataloader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
val_dataloader = DataLoader(dataset = val_dataset, batch_size = batch_size, shuffle = False)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [19]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, classes):
        super(Net, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),

            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),

            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),

            nn.Linear(128 * 4 * 4, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),

            nn.Linear(256, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [20]:
net = Net(classes = 10)

In [21]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params = net.parameters(), lr = 0.001, momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)

In [22]:
import torch
import os
from tqdm import tqdm

epochs = 1
model_save = 1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net.to(device)
criterion.to(device)

for epoch in range(epochs):
    net.train()
    running_loss = 0
    
    for inputs, labels in tqdm(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()

    net.eval()
    validation_loss = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = net(inputs)

            loss = criterion(outputs, labels)

            validation_loss += loss.item()

    scheduler.step()
    
    if (epoch + 1) % model_save == 0:
        torch.save(net.state_dict(), os.path.join('weights', f'{epoch + 1}.pth'))

    print('Epoch', epoch + 1)
    print('Training Loss:', running_loss / len(train_dataloader))
    print('Validation Loss:', validation_loss / len(val_dataloader))
    print()

100%|██████████| 157/157 [00:02<00:00, 60.81it/s]

Epoch 1
Training Loss: 1.5440288827896118
Validation Loss: 1.2397951077503764



In [23]:
import torch
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

y_true = torch.LongTensor([]).to(device)
y_pred = torch.LongTensor([]).to(device)

net.load_state_dict(torch.load('weights/20.pth'))

net.eval()

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = net(inputs)

        _, top_1_output = torch.max(outputs.data, dim=1)
        y_true = torch.cat((y_true, labels))
        y_pred = torch.cat((y_pred, top_1_output))

y_true = y_true.cpu()
y_pred = y_pred.cpu()

acc = accuracy_score(y_true, y_pred)
b_acc = balanced_accuracy_score(y_true, y_pred)
f1_mi = f1_score(y_true, y_pred, average='micro')
f1_ma = f1_score(y_true, y_pred, average='macro')
f1_w = f1_score(y_true, y_pred, average='weighted')
        
print(acc)
print(b_acc)
print(f1_mi)
print(f1_ma)
print(f1_w)

FileNotFoundError: [Errno 2] No such file or directory: 'weights/20.pth'

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
print(f'Classification Report:\n{cr}')
disp.plot()
plt.title('Confusion Matrix')
plt.show()